In [16]:
import nsepy as ns
import pandas as pd
import math

from datetime import datetime

import numpy as np
from pandas import Series, DataFrame
from pandas.plotting import scatter_matrix
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

from datetime import datetime 
from datetime import timedelta
from textblob import TextBlob

In [17]:
sbin = ns.get_history(symbol='INFY',
                    start=datetime(2015,1,1), 
                    end=datetime(2020,1,10))

sbin.tail()

,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
Date,,,,,,,,,,,,,,
2020-01-06,INFY,EQ,746.00,746.10,753.8,736.5,738.5,738.85,743.51,6519403,4.847237e+14,129992,2871332,0.4404
2020-01-07,INFY,EQ,738.85,738.00,742.6,725.1,729.3,727.90,734.63,7899363,5.803120e+14,189680,3976564,0.5034
2020-01-08,INFY,EQ,727.90,724.95,729.0,708.3,718.9,718.20,717.84,8639443,6.201708e+14,232857,3579984,0.4144
2020-01-09,INFY,EQ,718.20,721.70,731.5,714.3,727.6,727.55,724.60,13521186,9.797455e+14,161064,7560512,0.5592
2020-01-10,INFY,EQ,727.55,740.45,742.9,730.5,740.0,738.15,737.69,9341405,6.891078e+14,145267,2641897,0.2828


In [18]:
df = pd.DataFrame(sbin)
df.head()

,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
Date,,,,,,,,,,,,,,
2015-01-01,INFY,EQ,1972.55,1968.95,1982.00,1956.9,1971.00,1974.40,1971.34,500691,9.870306e+13,14908,258080,0.5154
2015-01-02,INFY,EQ,1974.40,1972.00,2019.05,1972.0,2017.95,2013.20,2003.25,1694580,3.394669e+14,54166,1249104,0.7371
2015-01-05,INFY,EQ,2013.20,2009.90,2030.00,1977.5,1996.00,1995.90,2004.59,2484256,4.979911e+14,82694,1830962,0.7370
2015-01-06,INFY,EQ,1995.90,1980.00,1985.00,1934.1,1965.10,1954.20,1954.82,2416829,4.724458e+14,108209,1772070,0.7332
2015-01-07,INFY,EQ,1954.20,1965.00,1974.75,1950.0,1966.05,1963.55,1962.59,1812479,3.557162e+14,62463,1317720,0.7270


In [19]:
df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2015-01-01,1968.95,1982.00,1956.9,1974.40,500691
2015-01-02,1972.00,2019.05,1972.0,2013.20,1694580
2015-01-05,2009.90,2030.00,1977.5,1995.90,2484256
2015-01-06,1980.00,1985.00,1934.1,1954.20,2416829
2015-01-07,1965.00,1974.75,1950.0,1963.55,1812479


In [20]:
df['HighLoad'] = (df['High'] - df['Close']) / df['Close'] * 100.0
df['Change'] = (df['Close'] - df['Open']) / df['Open'] * 100.0
df = df[['Close', 'HighLoad', 'Change', 'Volume']]

In [21]:
df['MA10'] = df['Close'].rolling(10).mean()
df['MA30'] = df['Close'].rolling(30).mean()
df['MA50'] = df['Close'].rolling(50).mean()

df['rets'] = df['Close'] / df['Close'].shift(1) - 1

In [22]:
df

,Close,HighLoad,Change,Volume,MA10,MA30,MA50,rets
Date,,,,,,,,
2015-01-01,1974.40,0.384927,0.276797,500691,NaN,NaN,NaN,NaN
2015-01-02,2013.20,0.290582,2.089249,1694580,NaN,NaN,NaN,0.019652
2015-01-05,1995.90,1.708502,-0.696552,2484256,NaN,NaN,NaN,-0.008593
2015-01-06,1954.20,1.576093,-1.303030,2416829,NaN,NaN,NaN,-0.020893
2015-01-07,1963.55,0.570395,-0.073791,1812479,NaN,NaN,NaN,0.004785
...,...,...,...,...,...,...,...,...
2020-01-06,738.85,2.023415,-0.971720,6519403,735.575,719.101667,708.644,-0.009584
2020-01-07,727.90,2.019508,-1.368564,7899363,734.765,720.095000,710.453,-0.014820
2020-01-08,718.20,1.503759,-0.931099,8639443,733.245,721.015000,711.840,-0.013326


In [24]:
    # Drop missing value
df.fillna(value=-99999, inplace=True)
    # We want to separate 1 percent of the data to forecast
forecast_out = int(math.ceil(0.01 * len(df)))
    # Separating the label here, we want to predict the AdjClose
forecast_col = 'Close'
df['label'] = df[forecast_col].shift(-forecast_out)
X = np.array(df.drop(['label'], 1))
    # Scale the X so that everyone can have the same distribution for linear regression
X = preprocessing.scale(X)
    # Finally We want to find Data Series of late X and early X (train) for model generation and evaluation
X_forecast = X[-forecast_out:]
X = X[:-forecast_out]
    # Separate label and identify it as y
y = np.array(df['label'])
y = y[:-forecast_out]

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [26]:
model = LinearRegression(n_jobs=-1)
model.fit(X_train, y_train)


    # KNN Regression
model_knn = KNeighborsRegressor(n_neighbors=2)
model_knn.fit(X_train, y_train)


    # Bayesian Ridge Regression
model_by = BayesianRidge()
model_by.fit(X_train, y_train)

BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, alpha_init=None,
              compute_score=False, copy_X=True, fit_intercept=True,
              lambda_1=1e-06, lambda_2=1e-06, lambda_init=None, n_iter=300,
              normalize=False, tol=0.001, verbose=False)

In [27]:
confidencereg = model.score(X_test, y_test)
confidence_model_knn = model_knn.score(X_test,y_test)
confidence_model_by = model_by.score(X_test,y_test)
    
reg = confidencereg * 100
knn = confidence_model_knn * 100
by = confidence_model_by * 100

score = " Regression {}\n KNN {}\n Bayesian {}\n ".format(reg,knn,by)

    # # results
score

' Regression 93.83543650449955\n KNN 93.76559651142085\n Bayesian 93.82113894317058\n '

In [28]:
forecast_reg = model.predict(X_forecast)
forecast_knn = model_knn.predict(X_forecast)
forecast_by = model_by.predict(X_forecast)

In [30]:
#Process all new columns data
df['Forecast_reg'] = np.nan

last_date = df.iloc[-1].name
last_unix = last_date
next_unix = last_unix + timedelta(days=1)

for i in forecast_reg:
    next_date = next_unix
    next_unix += timedelta(days=1)
    df.loc[next_date] = [np.nan for _ in range(len(df.columns))]
    df['Forecast_reg'].loc[next_date] = i

In [31]:
df

,Close,HighLoad,Change,Volume,MA10,MA30,MA50,rets,label,Forecast_reg
Date,,,,,,,,,,
2015-01-01,1974.40,0.384927,0.276797,500691.0,-99999.0,-99999.0,-99999.0,-99999.000000,2124.20,NaN
2015-01-02,2013.20,0.290582,2.089249,1694580.0,-99999.0,-99999.0,-99999.0,0.019652,2169.90,NaN
2015-01-05,1995.90,1.708502,-0.696552,2484256.0,-99999.0,-99999.0,-99999.0,-0.008593,2196.45,NaN
2015-01-06,1954.20,1.576093,-1.303030,2416829.0,-99999.0,-99999.0,-99999.0,-0.020893,2215.05,NaN
2015-01-07,1963.55,0.570395,-0.073791,1812479.0,-99999.0,-99999.0,-99999.0,0.004785,2136.50,NaN
...,...,...,...,...,...,...,...,...,...,...
2020-01-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,781.303975
2020-01-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,769.409568
2020-01-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,758.687556
